**Import Libs**

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

**Load Dataset**

In [0]:
data_frame = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Content-Based Recommender System Data/fv-tf-idf-scores.csv', header=None, sep=',')

In [3]:
header = []

for i in range(len(data_frame.columns)-1):
  header.append(str(i))

header.append('target')
data_frame.columns = header

data_frame.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,target
0,0.613706,0.613706,0.000000,0.000000,0.306853,0.306853,0.306853,0.000000,0.000000,0.613706,2.147970,0.306853,0.000000,0.306853,0.306853,0.613706,0.306853,0.613706,0.306853,0.920558,0.613706,0.000000,0.000000,0.306853,0.000000,0.306853,0.613706,0.000000,0.000000,0.306853,0.000000,0.306853,0.306853,0.306853,0.920558,0.613706,0.000000,0.306853,0.306853,0.306853,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.306853,0.306853,0.306853,0.306853,0.000000,0.306853,0.000000,0.613706,0.920558,0.613706,0.000000,0.000000,0.306853,0.000000,0.000000,1.227411,1.534264,0.306853,0.613706,0.306853,0.306853,0.306853,0.306853,0.306853,0.306853,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.306853,0.000000,0.306853,0.306853,0.306853,0.000000,0.306853,0.306853,0.306853,0.000000,0.306853,0.306853,0.613706,0.306853,0.306853,0.613706,0.613706,0.306853,0.306853,0.000000,0.000000,0.306853,0.306853,0.000000,0.000000,0.000000,0.000000,0.306853,0.306853,0.000000,0.306853,0.000000,0.000000,0.000000,0.000000,0.000000,0.920558,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.306853,0.000000,0.000000,0.000000,0.000000,0.000000,0.306853,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [4]:
x = data_frame.drop('target',axis=1)
y = data_frame['target']

x_train_df, x_test_df, y_train_df, y_test_df = train_test_split(x, y, test_size=0.2)

x_train = x_train_df.to_numpy()
x_test = x_test_df.to_numpy()
y_train = y_train_df.to_numpy()
y_test = y_test_df.to_numpy()

print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

(12, 500) (3, 500)
(12,) (3,)


**Hyperparameters**

In [0]:
num_features = 500
num_labels = 5
learning_rate = 0.001
batch_size = 3
training_epochs = 10
display_step = 1

**One-Hot Encoding**

In [6]:
def to_onehot(y):
  data = np.zeros((num_labels))
  data[y] = 1
  return data

x_train_one_hot_encoded = np.reshape(x_train, (-1, num_features))
x_test_one_hot_encoded = np.reshape(x_test, (-1, num_features))

y_train_one_hot_encoded = np.array([to_onehot(y) for y in y_train])
y_test_one_hot_encoded = np.array([to_onehot(y) for y in y_test])

print(x_train_one_hot_encoded.shape, x_test_one_hot_encoded.shape)
print(y_train_one_hot_encoded.shape, y_test_one_hot_encoded.shape)

(12, 500) (3, 500)
(12, 5) (3, 5)


**Model**

In [0]:
x = tf.placeholder(tf.float32, shape = [None, num_features])
y = tf.placeholder(tf.float32, shape = [None, num_labels])

weight = tf.Variable(tf.zeros([num_features, num_labels]))
bias = tf.Variable(tf.zeros([num_labels]))

pred = tf.nn.softmax(tf.matmul(x, weight) + bias)
cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices = 1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

init = tf.global_variables_initializer()

In [0]:
def get_accuracy(predictions, labels):
  correctly_predicted = np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
  accuracy = (100.0 * correctly_predicted) / predictions.shape[0]
  return accuracy

In [9]:
with tf.Session() as sess:
  # Run the initializer
  sess.run(init)
  
  # Training cycle
  for epoch in range(training_epochs):
    avg_cost = 0.0
    avg_acc = 0.0
    total_batch = len(x_train_one_hot_encoded)//batch_size
    
    for i in range(total_batch):
      batch_x = x_train_one_hot_encoded[i:i+1*batch_size]
      batch_y = y_train_one_hot_encoded[i:i+1*batch_size]
            
      _, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
      avg_cost += c/total_batch
            
      pred_y = sess.run(pred, feed_dict={x: batch_x})
      acc = get_accuracy(pred_y, batch_y)
      avg_acc += acc/total_batch
            
            
      if (epoch+1) % display_step == 0:
        tc = sess.run(cost, feed_dict={x: x_test_one_hot_encoded, y: y_test_one_hot_encoded})
        pred_y = sess.run(pred, feed_dict={x: x_test_one_hot_encoded})
        ta = get_accuracy(pred_y, y_test_one_hot_encoded)
            
        print("Epoch: {:2.0f} - Cost: {:1.5f} - Acc: {:0.5f} - Test Cost: {:0.5f} - Test Acc: {:0.5f}".format(
            epoch+1, avg_cost, avg_acc, tc, ta))
            
  print("Optimization Finshed")

  # Test model
  correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
  # Calculate accuracy
  acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  print("Test Accuracy:", acc.eval({x: x_test_one_hot_encoded, y: y_test_one_hot_encoded}))

Epoch:  1 - Cost: 0.40236 - Acc: 25.00000 - Test Cost: 1.60853 - Test Acc: 66.66667
Epoch:  1 - Cost: 0.80357 - Acc: 50.00000 - Test Cost: 1.60749 - Test Acc: 66.66667
Epoch:  1 - Cost: 1.20360 - Acc: 75.00000 - Test Cost: 1.60667 - Test Acc: 66.66667
Epoch:  1 - Cost: 1.60528 - Acc: 100.00000 - Test Cost: 1.60675 - Test Acc: 66.66667
Epoch:  2 - Cost: 0.39895 - Acc: 25.00000 - Test Cost: 1.60585 - Test Acc: 66.66667
Epoch:  2 - Cost: 0.79601 - Acc: 50.00000 - Test Cost: 1.60482 - Test Acc: 66.66667
Epoch:  2 - Cost: 1.19180 - Acc: 75.00000 - Test Cost: 1.60400 - Test Acc: 33.33333
Epoch:  2 - Cost: 1.59212 - Acc: 100.00000 - Test Cost: 1.60409 - Test Acc: 66.66667
Epoch:  3 - Cost: 0.39559 - Acc: 25.00000 - Test Cost: 1.60320 - Test Acc: 33.33333
Epoch:  3 - Cost: 0.78856 - Acc: 50.00000 - Test Cost: 1.60217 - Test Acc: 33.33333
Epoch:  3 - Cost: 1.18019 - Acc: 75.00000 - Test Cost: 1.60137 - Test Acc: 33.33333
Epoch:  3 - Cost: 1.57914 - Acc: 100.00000 - Test Cost: 1.60146 - Test Acc